In [85]:
import datetime as dt
import pandas as pd
import numpy as np
import math
from functools import reduce

In [86]:
# Set Peak
peak_month = [3,4,5,6,7,8,9,10,11,12,1,2]
peak_arr = [95,95,94,90,91,72,66,87,91,85,100,95]

peak_year = 2020
peak_date = list()
peak_df = pd.DataFrame(columns=["peak (kW)"])

for idx,_ in enumerate(peak_month):
    peak_date.append(
        dt.datetime(
            peak_year,
            _,
            1
        )
    )
    peak_df = peak_df.append({
        "peak (kW)": peak_arr[idx] 
    },ignore_index=True)
    
peak_df.index = peak_date
peak_df

,peak (kW)
2020-03-01,95
2020-04-01,95
2020-05-01,94
2020-06-01,90
2020-07-01,91
2020-08-01,72
2020-09-01,66
2020-10-01,87
2020-11-01,91
2020-12-01,85


In [87]:
# set household
household_name = ["{}01 호".format(_) for _ in range(1, 11)]
household_kwh = [150,180,220,210,310,300,270,190,250,260]
household_df = pd.DataFrame(columns=[
    "name",
    "usage (kWh)"
])

for idx, kwh in enumerate(household_kwh):
    household_df = household_df.append({
        "name": household_name[idx],
        "usage (kWh)": kwh
    }, ignore_index=True)

household_df

,name,usage (kWh)
0,101 호,150
1,201 호,180
2,301 호,120
3,401 호,210
4,501 호,310
5,601 호,210
6,701 호,270
7,801 호,190
8,901 호,250
9,1001 호,210


# 요금설정

In [88]:
# set kepco fee
KEPCO_FEE_STATIC = {
    "환경비용차감": -5,
    "기후환경요금": 5.3,
    "연료비조정액": -3,
    "필수사용량 보장공제 (월 1,000 kWh 초과), 저압": 709.5,
    "필수사용량 보장공제 (월 1,000 kWh 초과), 고압": 574.6,
}
KEPCO_FEE_DF = pd.DataFrame(columns=['name', 'fee', 'unit'])
for key in KEPCO_FEE_STATIC.keys():
    KEPCO_FEE_DF = KEPCO_FEE_DF.append({
        "name": key,
        "fee": KEPCO_FEE_STATIC[key],
        "unit": 'kWh'
    }, ignore_index=True)
    
KEPCO_FEE_STATIC_II = {
    "필수사용량 보장공제 (월 200 kWh 이하), 저압": 4000,
    "필수사용량 보장공제 (월 200 kWh 이하), 고압": 2500,
}
for key in KEPCO_FEE_STATIC_II.keys():
    KEPCO_FEE_DF = KEPCO_FEE_DF.append({
        "name": key,
        "fee": KEPCO_FEE_STATIC_II[key],
        "unit": None
    }, ignore_index=True)

KEPCO_FEE_DF

,name,fee,unit
0,환경비용차감,-5,kWh
1,기후환경요금,5.3,kWh
2,연료비조정액,-3,kWh
3,"필수사용량 보장공제 (월 1,000 kWh 초과), 저압",709.5,kWh
4,"필수사용량 보장공제 (월 1,000 kWh 초과), 고압",574.6,kWh
5,"필수사용량 보장공제 (월 200 kWh 이하), 저압",4000,None
6,"필수사용량 보장공제 (월 200 kWh 이하), 고압",2500,None


In [89]:
# 주택용전력 (저압) setting
fee_dict = dict()

household_high_pressure_fee = {
    "type":["주택용 고압" for _ in range(0,12)],
    "max kWh": reduce(lambda acc, cur: acc + [200,400,float("inf")], range(0,3),[]) \
        + [300, 450, float("inf")],
    "basic": reduce(lambda acc, cur: acc + [730, 1260, 6060], range(0,4),[]),
    "fee": reduce(lambda acc, cur: acc + [73.3, 142.3, 210.6], range(0,4),[]),
    "unit":["kWh" for _ in range(0,12)],
    "season":["spring" for _ in range(0,3)] + ["autumn" for _ in range(0,3)] + \
        ["winter" for _ in range(0,3)] + ["summer" for _ in range(0,3)]
}

household_high_pressure_fee_df = pd.DataFrame(household_high_pressure_fee,
                      columns=['type','max kWh', 'basic', 'fee', 'unit','season'])

fee_dict['주택용전력 (고압)'] = household_high_pressure_fee_df

# 전체통합
fee_df = pd.DataFrame(columns=['type','max kWh', 'basic', 'fee', 'unit','season'])
for _ in fee_dict.values():
    fee_df = pd.concat([fee_df, _], ignore_index=True)
    
fee_df

,type,max kWh,basic,fee,unit,season
0,주택용 고압,200.0,730,73.3,kWh,spring
1,주택용 고압,400.0,1260,142.3,kWh,spring
2,주택용 고압,inf,6060,210.6,kWh,spring
3,주택용 고압,200.0,730,73.3,kWh,autumn
4,주택용 고압,400.0,1260,142.3,kWh,autumn
5,주택용 고압,inf,6060,210.6,kWh,autumn
6,주택용 고압,200.0,730,73.3,kWh,winter
7,주택용 고압,400.0,1260,142.3,kWh,winter
8,주택용 고압,inf,6060,210.6,kWh,winter
9,주택용 고압,300.0,730,73.3,kWh,summer


# 단일계약
- 세대부, 공용부 : 주택용고압요금
- Season : Winter
- Calc Month : 2

In [95]:
APT_meter_kwh = 4000
household_count = len(household_df)

print("전체 전기 사용량 : {}kWh / 세대 수 : {}".format(APT_meter_kwh, household_count))

전체 전기 사용량 : 4000kWh / 세대 수 : 10


In [96]:
management_bill_df = pd.DataFrame(columns=["전체 전기 사용량 (kWh)", "세대 수", "평균 사용량 (kWh)",
                               "기본요금", "전력량요금", "기후환경요금", "연료비조정액",
                               "전기요금계", "부가세", "4사 5입", "전력산업기반기금", "전력산업기반기금 (절사)", 
                                "청구금액", "청구금액 (절사)"])
fees = list()
fee_dict = dict()

fee_dict['전체 전기 사용량 (kWh)'] = APT_meter_kwh
fee_dict['세대 수'] = household_count

# 평균 사용량
mean_usage = APT_meter_kwh / household_count
fee_dict['평균 사용량 (kWh)'] = mean_usage

sel_fee_df = fee_df[
    (fee_df['type'] == "주택용 고압") &\
    (fee_df['season'] == 'winter')
].copy()

# 기본요금
basic_fee = int(sel_fee_df[
    sel_fee_df['max kWh'] >= mean_usage 
]['basic'].values[0] * household_count)
fee_dict['기본요금'] = basic_fee
fees.append(basic_fee)

# 전력량요금
electricity_rate = 0
usage = mean_usage
step_bak = 0
idx = 0
while usage > 0:
    progressive_tax = sel_fee_df['fee'].iloc[idx]
    max_kwh = sel_fee_df['max kWh'].iloc[idx]

    step_kwh = usage if usage < max_kwh else max_kwh - step_bak
    electricity_rate += int(progressive_tax * step_kwh * 10)
    
    usage -= step_kwh
    step_bak = max_kwh
    idx += 1
fee_dict['전력량요금'] = electricity_rate
fees.append(electricity_rate)

# 기후환경요금
unit = KEPCO_FEE_STATIC['기후환경요금']
env_fee = int(APT_meter_kwh * unit)
fee_dict['기후환경요금'] = env_fee
fees.append(env_fee)

# 연료비조정액
unit = KEPCO_FEE_STATIC['연료비조정액']
fuel_cost_adjustment = int(APT_meter_kwh * unit)
fee_dict['연료비조정액'] = fuel_cost_adjustment
fees.append(fuel_cost_adjustment)

# 전기요금계
all_fee = sum(fees)
fee_dict['전기요금계'] = all_fee

VAT = all_fee * 0.1
VAT_process = int(round(VAT))
infra_fund = all_fee * 0.037
infra_fund_process = int(math.floor(infra_fund / 10) * 10)

fee_dict["부가세"] = VAT
fee_dict["4사 5입"] = VAT_process
fee_dict["전력산업기반기금"] = infra_fund
fee_dict["전력산업기반기금 (절사)"] = infra_fund_process

bill_fee = all_fee + VAT_process + infra_fund_process
bill_fee_process = int(math.floor(bill_fee / 10) * 10)

fee_dict["청구금액"] = bill_fee
fee_dict["청구금액 (절사)"] = bill_fee_process
    
management_bill_df = management_bill_df.append(
    pd.Series(
        fee_dict,
        name="관리사무소 청구금액"
    ))
management_bill_df

,전체 전기 사용량 (kWh),세대 수,평균 사용량 (kWh),기본요금,전력량요금,기후환경요금,연료비조정액,전기요금계,부가세,4사 5입,전력산업기반기금,전력산업기반기금 (절사),청구금액,청구금액 (절사)
관리사무소 청구금액,4000.0,10.0,400.0,12600.0,431200.0,21200.0,-12000.0,453000.0,45300.0,45300.0,16761.0,16760.0,515060.0,515060.0


In [97]:
household_bill_df = pd.DataFrame(columns=["사용량 (kWh)","기본요금","전력량요금","기후환경요금",
                                          "연료비조정액","필수사용량 보장공제","전기요금계", "부가세", 
                                          "4사 5입", "전력산업기반기금", "전력산업기반기금 (절사)", 
                                          "청구금액", "청구금액 (절사)"])

for household_idx in range(0, len(household_df)):
    household = household_df.iloc[household_idx]
    sel_fee_df = fee_df[
        (fee_df['type'] == "주택용 고압") &\
        (fee_df['season'] == 'winter')
    ].copy()

    fees = list()
    fee_dict = dict()

    # 기본요금
    basic_fee = sel_fee_df[
        sel_fee_df['max kWh'] >= household['usage (kWh)']
    ]['basic'].iloc[0]
    basic_fee
    fees.append(basic_fee)
    fee_dict['기본요금'] = basic_fee

    # 전력량요금
    electricity_rate = 0
    fix_usage = household['usage (kWh)']
    fee_dict["사용량 (kWh)"] = fix_usage
    usage = household['usage (kWh)']
    step_bak = 0
    idx = 0
    while usage > 0:
        progressive_tax = sel_fee_df['fee'].iloc[idx]
        max_kwh = sel_fee_df['max kWh'].iloc[idx]

        step_kwh = usage if usage < max_kwh else max_kwh - step_bak
        electricity_rate += int(progressive_tax * step_kwh)

        usage -= step_kwh
        step_bak = max_kwh
        idx += 1
    fees.append(electricity_rate)
    fee_dict['전력량요금'] = electricity_rate

    # 기후환경요금
    unit = KEPCO_FEE_STATIC['기후환경요금']
    env_fee = int(fix_usage * unit)
    fees.append(env_fee)
    fee_dict['기후환경요금'] = env_fee

    # 연료비조정액
    unit = KEPCO_FEE_STATIC['연료비조정액']
    fuel_cost_adjustment = int(fix_usage * unit)
    fees.append(fuel_cost_adjustment)
    fee_dict['연료비조정액'] = fuel_cost_adjustment

    # 필수사용량 보장공제 여부
    is_deduction = True if (fix_usage > 1000) \
                    or (fix_usage <= 200) else False
    if is_deduction:
        unit = KEPCO_FEE_STATIC['필수사용량 보장공제 (월 1,000 kWh 초과), 저압'] if (fix_usage > 1000) \
                else KEPCO_FEE_STATIC_II["필수사용량 보장공제 (월 200 kWh 이하), 고압"]
        deduction = (unit * (fix_usage - 1000) if (fix_usage > 1000) \
                else unit) * -1
        fees.append(deduction)
    fee_dict['필수사용량 보장공제'] = deduction if is_deduction else 0

    # 전기요금계
    all_fee = sum(fees)
    fee_dict["전기요금계"] = all_fee
    
    VAT = all_fee * 0.1
    VAT_process = round(VAT)
    infra_fund = all_fee * 0.037
    infra_fund_process = math.floor(infra_fund / 10) * 10
    
    fee_dict["부가세"] = VAT
    fee_dict["4사 5입"] = VAT_process
    fee_dict["전력산업기반기금"] = infra_fund
    fee_dict["전력산업기반기금 (절사)"] = infra_fund_process
    
    bill_fee = all_fee + VAT_process + infra_fund_process
    bill_fee_process = math.floor(bill_fee / 10) * 10
    fee_dict["청구금액"] = bill_fee
    fee_dict["청구금액 (절사)"] = bill_fee_process

    household_bill_df = household_bill_df.append(pd.Series(fee_dict, name=household['name']))

households_dict = dict()
for _ in household_bill_df:
    households_dict[_] = household_bill_df[_].sum()
household_bill_df = household_bill_df.append(
                    pd.Series(
                        households_dict,
                        name="세대 전체 요금 합산 {} 월".format(2)
                    ))    
household_bill_df

,사용량 (kWh),기본요금,전력량요금,기후환경요금,연료비조정액,필수사용량 보장공제,전기요금계,부가세,4사 5입,전력산업기반기금,전력산업기반기금 (절사),청구금액,청구금액 (절사)
101 호,150.0,730.0,10995.0,795.0,-450.0,-2500.0,9570.0,957.0,957.0,354.090,350.0,10877.0,10870.0
201 호,180.0,730.0,13194.0,954.0,-540.0,-2500.0,11838.0,1183.8,1184.0,438.006,430.0,13452.0,13450.0
301 호,120.0,730.0,8796.0,636.0,-360.0,-2500.0,7302.0,730.2,730.0,270.174,270.0,8302.0,8300.0
401 호,210.0,1260.0,16083.0,1113.0,-630.0,0.0,17826.0,1782.6,1783.0,659.562,650.0,20259.0,20250.0
501 호,310.0,1260.0,30313.0,1643.0,-930.0,0.0,32286.0,3228.6,3229.0,1194.582,1190.0,36705.0,36700.0
601 호,210.0,1260.0,16083.0,1113.0,-630.0,0.0,17826.0,1782.6,1783.0,659.562,650.0,20259.0,20250.0
701 호,270.0,1260.0,24621.0,1431.0,-810.0,0.0,26502.0,2650.2,2650.0,980.574,980.0,30132.0,30130.0
801 호,190.0,730.0,13927.0,1007.0,-570.0,-2500.0,12594.0,1259.4,1259.0,465.978,460.0,14313.0,14310.0
901 호,250.0,1260.0,21775.0,1325.0,-750.0,0.0,23610.0,2361.0,2361.0,873.570,870.0,26841.0,26840.0
1001 호,210.0,1260.0,16083.0,1113.0,-630.0,0.0,17826.0,1782.6,1783.0,659.562,650.0,20259.0,20250.0


In [98]:
common_columns = list(set(household_bill_df.columns) & set(management_bill_df.columns))
common_columns = ['기본요금', '전력량요금', '기후환경요금', '연료비조정액', '전기요금계', '부가세', '4사 5입', 
                  '전력산업기반기금', '전력산업기반기금 (절사)', '청구금액', '청구금액 (절사)']
public_bill_df = pd.DataFrame(columns=common_columns)
public_bill_df.loc['공동사용설비요금 {} 월'.format(2)] = \
            ["-" for _ in range(0, len(common_columns) - 1)] + [
                management_bill_df['청구금액 (절사)'].values[0]\
                -\
                household_bill_df.loc["세대 전체 요금 합산 {} 월".format(2)]['청구금액 (절사)']
            ]

bill_df = pd.concat([
    household_bill_df[common_columns],
    public_bill_df,
    management_bill_df[common_columns]
])
bill_df

,기본요금,전력량요금,기후환경요금,연료비조정액,전기요금계,부가세,4사 5입,전력산업기반기금,전력산업기반기금 (절사),청구금액,청구금액 (절사)
101 호,730.0,10995.0,795.0,-450.0,9570.0,957.0,957.0,354.09,350.0,10877.0,10870.0
201 호,730.0,13194.0,954.0,-540.0,11838.0,1183.8,1184.0,438.006,430.0,13452.0,13450.0
301 호,730.0,8796.0,636.0,-360.0,7302.0,730.2,730.0,270.174,270.0,8302.0,8300.0
401 호,1260.0,16083.0,1113.0,-630.0,17826.0,1782.6,1783.0,659.562,650.0,20259.0,20250.0
501 호,1260.0,30313.0,1643.0,-930.0,32286.0,3228.6,3229.0,1194.582,1190.0,36705.0,36700.0
601 호,1260.0,16083.0,1113.0,-630.0,17826.0,1782.6,1783.0,659.562,650.0,20259.0,20250.0
701 호,1260.0,24621.0,1431.0,-810.0,26502.0,2650.2,2650.0,980.574,980.0,30132.0,30130.0
801 호,730.0,13927.0,1007.0,-570.0,12594.0,1259.4,1259.0,465.978,460.0,14313.0,14310.0
901 호,1260.0,21775.0,1325.0,-750.0,23610.0,2361.0,2361.0,873.57,870.0,26841.0,26840.0
1001 호,1260.0,16083.0,1113.0,-630.0,17826.0,1782.6,1783.0,659.562,650.0,20259.0,20250.0


In [99]:
public_bill = bill_df.loc['공동사용설비요금 {} 월'.format(2)]['청구금액 (절사)'] / household_count
bill_df['공동전기사용료'] = [public_bill for _ in range(0, household_count)] + ["-", "-", "-"]
bill_df['최종청구금액'] = [household_bill_df.iloc[_]['청구금액 (절사)'] + public_bill for _
                        in range(household_count)] + ["-", "-", "-"]
bill_df

,기본요금,전력량요금,기후환경요금,연료비조정액,전기요금계,부가세,4사 5입,전력산업기반기금,전력산업기반기금 (절사),청구금액,청구금액 (절사),공동전기사용료,최종청구금액
101 호,730.0,10995.0,795.0,-450.0,9570.0,957.0,957.0,354.09,350.0,10877.0,10870.0,31371.0,42241.0
201 호,730.0,13194.0,954.0,-540.0,11838.0,1183.8,1184.0,438.006,430.0,13452.0,13450.0,31371.0,44821.0
301 호,730.0,8796.0,636.0,-360.0,7302.0,730.2,730.0,270.174,270.0,8302.0,8300.0,31371.0,39671.0
401 호,1260.0,16083.0,1113.0,-630.0,17826.0,1782.6,1783.0,659.562,650.0,20259.0,20250.0,31371.0,51621.0
501 호,1260.0,30313.0,1643.0,-930.0,32286.0,3228.6,3229.0,1194.582,1190.0,36705.0,36700.0,31371.0,68071.0
601 호,1260.0,16083.0,1113.0,-630.0,17826.0,1782.6,1783.0,659.562,650.0,20259.0,20250.0,31371.0,51621.0
701 호,1260.0,24621.0,1431.0,-810.0,26502.0,2650.2,2650.0,980.574,980.0,30132.0,30130.0,31371.0,61501.0
801 호,730.0,13927.0,1007.0,-570.0,12594.0,1259.4,1259.0,465.978,460.0,14313.0,14310.0,31371.0,45681.0
901 호,1260.0,21775.0,1325.0,-750.0,23610.0,2361.0,2361.0,873.57,870.0,26841.0,26840.0,31371.0,58211.0
1001 호,1260.0,16083.0,1113.0,-630.0,17826.0,1782.6,1783.0,659.562,650.0,20259.0,20250.0,31371.0,51621.0
